In [ ]:
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install -U colabgymrender
!pip install -U moviepy==0.2.3.5
!pip install imageio==2.4.1
!pip install --upgrade AutoROM
!AutoROM --accept-license
!pip install gym[classic_control] > /dev/null 2>&1

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import gym
from colabgymrender.recorder import Recorder
directory = './video'


## Introducing the Environment

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1nH4ia3YaOF455jcrfXNzKxTNpMQl8gdP?usp=sharing)

### CartPole-V0

CartPole-V0 is a classic control problem in the field of reinforcement learning. The task is to balance a pole on a cart by moving the cart left or right. The environment consists of a cart that can move horizontally along a track, and a pole that is attached to the cart by a hinge. The pole can rotate freely around the hinge, and the goal is to keep the pole balanced by moving the cart to keep the pole upright.

The observation space consists of four variables: the horizontal position and velocity of the cart, and the angle and angular velocity of the pole. The action space consists of two discrete actions: move the cart to the left, or move the cart to the right. The episode ends when the pole falls beyond a certain angle, or the cart moves too far from the center.

In [ ]:
env = gym.make('CartPole-v0')
env = Recorder(env, directory)

obs = env.reset()
for i in range(500):
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)

    if done:
        break

env.play()

# Part 1: Policy Gradient

In this assignment, we will implement the Policy Gradient algorithm. This algorithm is a popular reinforcement learning algorithm that can be used for both discrete and continuous action spaces. 

## Overview

The Policy Gradient algorithm uses a neural network to learn a policy that maps states to actions. The algorithm collects a set of trajectories, which are used to update the policy network. 

The goal in reinforcement learning to maximize the performance, i.e. find a policy $\pi_{\theta}$ that induce a distribution over trajectories $P_\theta(\tau)$ such that when we sample trajectories $\tau$ from it, the expected reward $r(\tau)$ is maximized. 
$$ J(\theta) = \mathbb{E}_{\tau \sim P_{\theta}}[r(\tau)] $$

each trajectory or rollout $\tau$ is of length $T$ can be defined as 

$$ P(\tau) = p(s_0) \pi_θ (a_0|s_0) \prod_{t=1}^{T-1} p (s_t|s_{t-1}, a_{t-1}) \pi_\theta (a_t | s_t)$$

and the reward of a trajectory is

$$ r(\tau) = \sum_{t=0}^{T-1} r(s_t, a_t)$$

The policy gradient theorem gives us a gradient for the objective:

$$ \nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim P_{\theta}}[\nabla_θ \log P_\theta (\tau) r(\tau)] $$

which in practice can be approximated from $N$ trajectories as follows:-

$$ \nabla_\theta J(\theta) \approx \frac{1}{N} \sum_{i=1}^{N} \Biggl( \sum_{t=0}^{T-1} \nabla_\theta \log \pi_\theta(a_{it}|s_{it})\Biggl) \Biggl(\sum_{t=0}^{T-1} r(s_{it}, a_{it}) \Biggl) $$

where $s_{it}$ and $a_{it}$ are the states and actions of trajectory $i$ and time $t$. 

Multiplying returns by a discount factor γ can serve as an incentive for the agent to prioritize more immediate rewards over those in the distant future. Additionally, it can help to decrease variability by reducing the potential for variance in future outcomes. 

Adjusting for the discounted return the gradient, the reward function $r(\tau)$ can be converted into this:


$$ r(\tau) = \sum_{t'=0}^{T-1} \gamma^{t'-1} r(s_{it'}, a_{it'}) $$

where $r_i$ is the reward at time step $i$, and $\gamma$ is the discount factor.

## Instructions

You will need to implement the following:

1. `PolicyNet` class - This class will define the policy network used in the Policy Gradient algorithm. 

2. `PolicyGradient` class - This class will define the Policy Gradient algorithm. You will need to modify the `train` method to collect trajectories and update the policy network using the Policy Gradient loss function. 

3. `evaluate` method - This method will evaluate the policy network by running multiple episodes. 

Follow the instructions below to implement each of these components.

### `PolicyNet` class

The `PolicyNet` class should define a neural network that takes in a state and outputs a probability distribution over the action space. The network should have the following architecture:

- Input layer: a fully-connected layer with `state_dim` input nodes and `hidden_dim` output nodes, followed by a ReLU activation function.

- Output layer: a fully-connected layer with `hidden_dim` input nodes and `action_dim` output nodes, followed by a ReLU activation function and then a softmax activation function.

You should use the `nn` module of PyTorch to define this network. 

In [ ]:
class PolicyNet(nn.Module):
    def __init__(self, state_dim: int, action_dim: int, hidden_dim: int):
        """Policy network for the REINFORCE algorithm.

        Args:
            state_dim (int): Dimension of the state space.
            action_dim (int): Dimension of the action space.
            hidden_dim (int): Dimension of the hidden layers.
        """
        super(PolicyNet, self).__init__()
        ## TODO: Implement the policy network for the REINFORCE algorithm here

    def forward(self, state: torch.Tensor):
        """Forward pass of the policy network.

        Args:
            state (torch.Tensor): State of the environment.

        Returns:
            x (torch.Tensor): Probabilities of the actions.
        """
        ## TODO: Implement the forward pass of the policy network here


### `PolicyGradient` class

The `PolicyGradient` class should define the Policy Gradient algorithm. You will need to modify the `train` method to collect trajectories and update the policy network using the Policy Gradient loss function. 

The `PolicyGradient` class should have the following methods:

- `__init__(self, env, policy_net: PolicyNet, reward_to_go: bool = False)`: Constructor method that initializes the environment and policy network.

- `select_action(self, state)`: Method that selects an action based on the policy network.

- `compute_loss(self, episode, gamma)`: Method that computes the loss given the episode which is a list of states, actions and rewards

- `update_policy(self, episodes, optimizer, gamma)`: Method that updates the policy network

- `train(self, num_outer_loop, num_episodes, gamma, lr)`: Method that trains the policy network using the Policy Gradient algorithm.

- `run_episode(self)`: Method that runs an episode (list of (state, action, reward) tuples) of the environment and returns the episode.

- `evaluate(self, num_episodes = 100, max_steps = 1000)`: Method that evaluates the policy network by running multiple episodes.

In [ ]:

class PolicyGradient:
    def __init__(self, env, policy_net: PolicyNet, reward_to_go: bool = False):
        """Policy gradient algorithm based on the REINFORCE algorithm.

        Args:
            env (gym.Env): Environment
            policy_net (PolicyNet): Policy network
            reward_to_go (bool): Whether to use returns or reward-to-go
        """
        self.env = Recorder(env, directory)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.policy_net = policy_net.to(self.device)
        self.reward_to_go = reward_to_go
        
    def select_action(self, state):
        """Select an action based on the policy network

        Args:
            state (np.ndarray): State of the environment

        Returns:
            action (int): Action to be taken
        """
        ## TODO: Implement the action selection here based on the policy network output probabilities
        ## Hint: Use torch.distributions.Categorical
    
    def compute_loss(self, episode, gamma = 0.99):
        """Compute the loss function for the REINFORCE algorithm

        Args:
            episode (list): List of tuples (state, action, reward). 
            gamma (float): Discount factor

        Returns:
            loss (torch.Tensor): Loss function
        """
        # Extract states, actions and rewards from the episode
        
        # Compute the discounted returns
        if not self.reward_to_go:
          ## TODO: Part 1: Compute the discounted returns here
          pass
        else:
          ## TODO: Part 2: Compute the discounted reward to go here
          pass


        ## TODO: Implement the loss function for the REINFORCE algorithm here based on the discounted returns and the log probabilities of the actions
        loss = None

        return loss
    
    def update_policy(self, episodes, optimizer, gamma):
        """Update the policy network using the batch of episodes

        Args:
            episodes (list): List of episodes
            optimizer (torch.optim): Optimizer
            gamma (float): Discount factor
        """
        # Compute the loss function for each episode
        losses = None
        
        # Compute the gradients and update the policy network
        pass
    
    def run_episode(self, render = False):
        """
        Run an episode of the environment and return the episode
        
        Returns:
            episode (list): List of tuples (state, action, reward)
        """
        state = self.env.reset()
        episode = []
        done = False
        while not done:
            action = self.select_action(state)
            next_state, reward, done, info = self.env.step(action)
            episode.append((state, action, reward))
            state = next_state
        if render:
          self.env.play()
        return episode

    def train(self, num_outer_loop, num_episodes, gamma, lr):
        """Train the policy network using the REINFORCE algorithm

        Args:
            num_outer_loop (int): Number of outerloops, i.e., calls to update_policy
            num_episodes (int): Number of episodes to collect in each iteration
            gamma (float): Discount factor
            lr (float): Learning rate
        """
        ## TODO: Implement the training loop for the REINFORCE algorithm here 
        ## using the update_policy function to update the policy network
    
    def evaluate(self, num_episodes = 100, max_steps = 1000):
        """Evaluate the policy network by running multiple episodes.

        Args:
            num_episodes (int): Number of episodes to run
            max_steps (int): Maximum number of steps in the episode
        Returns:
            average_reward (float): Average reward over the episodes
        """
        ## TODO: Implement the evaluation loop for the REINFORCE algorithm here by running multiple episodes and averaging the returns

In [ ]:
env = gym.make("CartPole-v0")
## TODO: Construct the policy network and the policy gradient algorithm here and train the policy network 


## Conclusion

In this part of the assignment, you have implemented the Policy Gradient algorithm. This algorithm is a popular reinforcement learning algorithm that can be used for both discrete and continuous action spaces. 

You have learned how to define a policy network and how to update the policy network using the Policy Gradient loss function. You have also learned how to train and evaluate the Policy Gradient algorithm.

# Part 2: Reward-to-Go

In this part of the assignment, we will modify the Policy Gradient algorithm to use the reward-to-go instead of the total discounted reward. 

To decrease the variance of the policy gradient, one approach is to utilize causality, which means that the policy cannot influence past reward. This results in a revised objective where the total returns only include those acquired after the policy is evaluated. These returns are considered a sample estimation of the Q function and are known as the "reward-to-go."

## Overview

The reward-to-go is defined as:

Without discounting:
$$ r(\tau) = \sum_{t'=t}^{T-1} r(s_{it'}, a_{it'}) $$

With discounting:
$$ r(\tau) = \sum_{t'=t}^{T-1} \gamma^{t'-1} r(s_{it'}, a_{it'}) $$

where $r_i$ is the reward at time step $i$. 

The updated policy gradient (with discounted reward-to-go) will look like this: 

$$ \nabla_\theta J(\theta) \approx \frac{1}{N} \sum_{i=1}^{N} \sum_{t=0}^{T-1} \Biggl(\nabla_\theta log \pi_\theta(a_{it}|s_{it}) \Biggl(\sum_{t'=t}^{T-1} \gamma^{t'-1} r(s_{it'}, a_{it'}) \Biggl) \Biggl)$$



## Instructions

You will need to modify the `PolicyGradient` class to use the reward-to-go instead of the total discounted return. 

Follow the instructions below to modify the `PolicyGradient` class:

1. Modify the `train` method to compute the reward-to-go for each time step.

2. Modify the `compute_loss` method to use the reward-to-go instead of the total discounted return.

In [ ]:
env = gym.make("CartPole-v0")
## TODO: Construct the policy network with and the policy gradient algorithm now with the reward to go functionality here and train the policy network 

In [ ]:
# Feel free to use the space below to run experiments and plots used in your writeup.


## Conclusion

In this part of the assignment, you have modified the Policy Gradient algorithm to use the reward-to-go instead of the total discounted return. 

You have learned how to compute the reward-to-go, and how to modify the Policy Gradient loss function to use the reward-to-go. 

# Part 3: Actor-Critic Policy Gradient (Extra Credit)

In this part of the assignment, we will implement the Actor-Critic Policy Gradient algorithm. This algorithm is an extension to the Policy Gradient algorithm, and it combines both value-based and policy-based methods. 

This is another way to reduce the variance of the model by subtracting a baseline (which is constant with respect to ${\tau}$ ) from the sum of returns. This modifies our original objective function as follows:-

$$ J(\theta) = \mathbb{E}_{\tau \sim \pi_{\theta}}[r(\tau) - b] $$

where b is the baseline

## Overview

The Actor-Critic Policy Gradient algorithm consists of two networks: the policy network and the value network. The policy network is used to select actions, while the value network is used to estimate the value of a state. 

In this part, we will construct a value function $V_ϕ^\pi$ that functions as a baseline that depends on the state. The purpose of this value function is to estimate the total future rewards that begin from a specific state.

The value function can be estimated as follows:-

$$ V_ϕ^\pi (s_t) \approx \sum_{t'= t}^{T-1} \mathbb{E}_{\pi_\theta} [r(s_{t'}, a_{t'})|s_t] $$


In each episode, the algorithm collects a set of trajectories, which are used to update the policy network and the value network. The update is done using the following loss functions:

$$ L_{\text{policy}} = -\sum_{t=0}^{T-1} \log \pi_{\theta}(a_t|s_t) A_t $$

$$ L_{\text{value}} = \frac{1}{2}\sum_{t=0}^{T} (V_{\phi}(s_t) - R_t)^2 $$

where $\pi_{\theta}$ is the policy network, $a_t$ is the action selected at time step $t$, $s_t$ is the state at time step $t$, $A_t$ is the advantage function, $V_{\phi}$ is the value network, and $R_t$ is the discounted return at time step $t$. 

The advantage function is defined as:

$$ A_t = Q_t - V_{\phi}^\pi(s_t) $$

where $Q_t$ is the estimated value of the state-action pair $(s_t, a_t)$.

and the final policy gradient now looks as follows:

$$ \nabla_\theta J(\theta) \approx \frac{1}{N} \sum_{i=1}^{N}  \sum_{t=0}^{T-1} \nabla_\theta log \pi_\theta(a_{it}|s_{it}) \Biggl( \Biggl(\sum_{t'=t}^{T-1} \gamma^{t'-1} r(s_{it'}, a_{it'}) \Biggl) - V_ϕ^\pi (s_{it}) \Biggl) $$

## Instructions

You will need to implement the following:

1. `ValueNet` class - This class will define the value network used in the Actor-Critic algorithm. 

2. `ActorCriticPolicyGradient` class - This class will define the Actor-Critic Policy Gradient algorithm. You will need to modify the `PolicyGradient` class to include the value network and to compute the loss using the Actor-Critic loss functions. 

3. `compute_loss` method - This method will compute the policy loss and value loss for a given episode. 

4. `update_policy` method - This method will update the policy network and value network using a batch of episodes. 

5. `train` method - This method will train the policy network and value network using the Actor-Critic algorithm. 

6. `evaluate` method - This method will evaluate the policy network by running multiple episodes. 

Follow the instructions below to implement each of these components.

### `ValueNet` class

The `ValueNet` class should define a neural network that takes in a state and outputs an estimate of the value of that state. The network should have the following architecture:

- Input layer: a fully-connected layer with `state_dim` input nodes and `hidden_dim` output nodes, followed by a ReLU activation function.

- Output layer: a fully-connected layer with `hidden_dim` input nodes and 1 output node.

You should use the `nn` module of PyTorch to define this network. 

In [ ]:
class ValueNet(nn.Module):
    def __init__(self, state_dim: int, hidden_dim: int):
        """Value network for the Actor-Critic algorithm.

        Args:
            state_dim (int): Dimension of the state space.
            hidden_dim (int): Dimension of the hidden layers.
        """
        super(ValueNet, self).__init__()
        ## TODO: Implement the value network for the REINFORCE with baseline algorithm here

    def forward(self, state: torch.Tensor):
        """Forward pass of the value network.

        Args:
            state (torch.Tensor): State of the environment.

        Returns:
            x (torch.Tensor): Estimated value of the state.
        """
        ## TODO: Implement the forward pass of the value network here


### `ActorCriticPolicyGradient` class

The `ActorCriticPolicyGradient` class should define the Actor-Critic Policy Gradient algorithm. You will need to modify the `PolicyGradient` class to include the value network and to compute the loss using the Actor-Critic loss functions. 

Only apply the **discounted reward-to-go** formulation here for calculating the rewards


### `compute_loss` method
The `compute_loss` method should compute the policy loss and value loss for a given episode. 

The policy loss is the negative sum of the log probabilities of the actions multiplied by the advantage function:

$$ L_{\text{policy}} = -\sum_{t=0}^{T} \log \pi_{\theta}(a_t|s_t) A_t $$

where $\pi_{\theta}$ is the policy network, $a_t$ is the action selected at time step $t$, $s_t$ is the state at time step $t$, and $A_t$ is the advantage function.

The value loss is the mean squared error between the estimated value of the state and the discounted return:

$$ L_{\text{value}} = \frac{1}{2}\sum_{t=0}^{T} (V_{\phi}(s_t) - R_t)^2 $$

where $V_{\phi}$ is the value network, and $R_t$ is the return at time step $t$.

You will need to modify the `compute_loss` method in the `ActorCriticPolicyGradient` class to compute these losses. 

In [ ]:

class ActorCriticPolicyGradient(PolicyGradient):
    def __init__(self, env, policy_net: PolicyNet, value_net: ValueNet):
        """Actor-Critic policy gradient algorithm.

        Args:
            env (gym.Env): Environment
            policy_net (PolicyNet): Policy network
            value_net (ValueNet): Value network
        """
        self.env = Recorder(env, directory)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.policy_net = policy_net.to(self.device)
        self.value_net = value_net.to(self.device)
            
    def compute_loss(self, episode, gamma = 0.99):
        """Compute the loss function for the Actor-Critic algorithm

        Args:
            episode (list): List of tuples (state, action, reward)
            gamma (float): Discount factor

        Returns:
            policy_loss (torch.Tensor): Policy loss function
            value_loss (torch.Tensor): Value loss function
        """
        # Extract states, actions and rewards from the episode

        # Compute the discounted reward-to-go

        policy_loss = None
        value_loss = None
        ## TODO: Implement the loss function for the REINFORCE algorithm here based on the discounted rewards and the log probabilities of the actions
        
        return policy_loss, value_loss
    
    def update_policy(self, episodes, optimizer, value_optimizer, gamma):
        """Update the policy network and value network using the batch of episodes

        Args:
            episodes (list): List of episodes
            optimizer (torch.optim): Optimizer for the policy network
            value_optimizer (torch.optim): Optimizer for the value network
            gamma (float): Discount factor
        """
        # TODO: Compute the loss function for each episode
        policy_losses = None
        value_losses = None
        
        # TODO: Compute the gradients and update the policy network and value network
        pass

    def train(self, num_outer_loop, num_episodes, gamma, lr):
        """Train the policy network using the REINFORCE algorithm

        Args:
            num_outer_loop (int): Number of outerloops i.e. iterations, i.e., calls to update_policy
            num_episodes (int): Number of episodes to collect in each iteration
            gamma (float): Discount factor
            lr (float): Learning rate
        """
        ## TODO: Implement the training of the policy network and value network using the Actor-Critic algorithm 

In [ ]:
env = gym.make("CartPole-v0")
## TODO: Construct the policy network, value networks and the policy gradient algorithm here and train the policy network and value networks

# Conclusion

In this assignment, we have implemented three variants of the Policy Gradient algorithm: 

1. The vanilla Policy Gradient algorithm, which uses the total discounted return as the target.

2. The Policy Gradient algorithm with reward-to-go, which uses the reward-to-go as the target.

3. The Actor-Critic Policy Gradient algorithm, which combines both value-based and policy-based methods. 

We have learned how to implement each of these algorithms using PyTorch and OpenAI Gym. We have also learned how to modify the loss function and update the network parameters to account for the different variants of the Policy Gradient algorithm.

Overall, the Policy Gradient algorithm is a powerful method for solving reinforcement learning problems where the action space is continuous and the environment is stochastic. The reward-to-go and Actor-Critic variants of the algorithm can improve performance and stability in some cases.